In [ ]:
import os
import pandas as pd
from translate import translate
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image

# 1- Build DataFrame with translated labels

dataset_path = r"/Users/ia_dev/Desktop/Work/archive/raw-img"
data = []

# Iterate through each folder in the dataset directory
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)

    # Check if the current path is actually a directory
if os.path.isdir(folder_path):
        # Get the label from the 'translate' dict (not shown), or default to folder name
        label = translate.get(folder, folder)

        # Iterate through files inside the specific category folder
        for file in os.listdir(folder_path):
            # Filter specifically for image formats
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(folder_path, file)
                # Store the full path and its corresponding label
                data.append([img_path, label])  
# Create a Pandas DataFrame to manage metadata
df = pd.DataFrame(data, columns=["image_path", "label"])

# Display the first few rows and distribution stats
print(df.head())
print("Total images:", len(df))
print(df['label'].value_counts())

# 2- Train / Test Split (90/10)

train_df, test_df = train_test_split(
    df,
    test_size=0.1,
    stratify=df['label'],
    random_state=42
)
print("Train size:", len(train_df))
print("Test size:", len(test_df))

# 3- Function to convert DataFrame to tf.data.Dataset

IMG_SIZE = 224
BATCH_SIZE = 32

def df_to_dataset(df, shuffle=True, batch_size=32):
# Extract file paths and convert labels from strings to integer codes
    paths = df['image_path'].values
    labels = df['label'].astype('category').cat.codes.values  # string -> integer
    # Create a basic TensorSliceDataset
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    # Internal function to load and preprocess images
    def process_path(path, label):
        # Read the raw file from disk
        img = tf.io.read_file(path)
        # Decode the JPEG format into a tensor (0-255 integers)
        img = tf.image.decode_jpeg(img, channels=3)
        # Resize image to the input size required by the model (224x224)
        img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
        # Normalize pixel values to range [0, 1] (crucial for convergence)
        img = img / 255.0
        return img, label

    # Apply the processing function in parallel using available CPU cores
    ds = ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

    # Shuffle the dataset to prevent the model from learning order-based patterns
if shuffle:
ds = ds.shuffle(buffer_size=len(df))

    # Group examples into batches
ds = ds.batch(batch_size)
    # Prefetch data to GPU memory while the CPU prepares the next batch (pipeline optimization)
ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)

return ds

# 4- Create TensorFlow Datasets

# Create the actual dataset objects ready for training
train_dataset = df_to_dataset(train_df, batch_size=BATCH_SIZE)
# Do not shuffle test data.
test_dataset = df_to_dataset(test_df, shuffle=False, batch_size=BATCH_SIZE)

# 5- Quick verification

# Take 1 batch from the training set to verify shapes and values
for images, labels in train_dataset.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    print("Example labels:", labels[:10].numpy())
    print("Min pixel:", images.numpy().min())
    print("Max pixel:", images.numpy().max())

# Visualize the first image
img = images[0].numpy()
Image.fromarray((img * 255).astype("uint8")).show()

SyntaxError: invalid non-printable character U+00A0 (1931629403.py, line 15)